In [ ]:
import sys
sys.path.append('../')

In [ ]:
import os

import h5py as h5
import numpy as np

from tqdm import tqdm

In [ ]:
base_dir = "/Volumes/Extreme_SSD/MegaDepth/scenes/"
scene = "0229"
features_dir = os.path.join(base_dir, scene, "features")
matches_dir = os.path.join(base_dir, scene, "matches")

In [ ]:
os.listdir(features_dir)

In [ ]:
super_features = h5.File(os.path.join(features_dir, "superpoint_max.h5"), "r")
super_matches = h5.File(os.path.join(matches_dir, "superpoint_max-superglue-netvlad-50.h5"), "r")

In [ ]:
loftr_features = h5.File(os.path.join(features_dir, "loftr-netvlad-20.h5"), "r")
loftr_matches = h5.File(os.path.join(matches_dir, "loftr-netvlad-20.h5"), "r")

In [ ]:
ensemble_features = {
    img: {
        "keypoints": np.concatenate(
            [super_features[img]["keypoints"], loftr_features[img]["keypoints"]], axis=0,
        ),
        "scores": np.concatenate(
            [super_features[img]["scores"], loftr_features[img]["score"]], axis=0
        ),
        "n_superpoint": super_features[img]["keypoints"].shape[0],
        "n_loftr": loftr_features[img]["keypoints"].shape[0],
    }
    for img in tqdm(super_features.keys())
}

In [ ]:
ens_kp_ds = h5.File(os.path.join(features_dir, "ensemble.h5"), "w")
for img in tqdm(ensemble_features.keys()):
    ens_kp_ds.create_group(img)
    for k in ensemble_features[img].keys():
        ens_kp_ds[img].create_dataset(k, data=ensemble_features[img][k])

ens_kp_ds.close()

In [ ]:
ensemble_features = h5.File(os.path.join(features_dir, "ensemble.h5"), "r")
features = ensemble_features[next(iter(ensemble_features.keys()))]
features.keys()

In [ ]:
ensemble_features.keys()

In [ ]:
ensemble_matches = {}

for img1 in tqdm(ensemble_features.keys()):
    ensemble_matches[img1] = {}
    for img2 in ensemble_features.keys():
        if img1 == img2:
            continue
        ensemble_matches[img1][img2] = {}

        # try:
        #     super_m = super_matches[img1][img2]
        # except KeyError:
        #     super_m = super_matches[img2][img1]

        # try:
        #     loftr_m = loftr_matches[img1][img2]
        # except KeyError:
        #     loftr_m = loftr_matches[img2][img1]

        if img1 in super_matches.keys() and img2 in super_matches[img1].keys():
            super_m = super_matches[img1][img2]
        elif img2 in super_matches.keys() and img1 in super_matches[img2].keys():
            super_m = super_matches[img2][img1]
        else:
            super_m = None

        if img1 in loftr_matches.keys() and img2 in loftr_matches[img1].keys():
            loftr_m = loftr_matches[img1][img2]
        elif img2 in loftr_matches.keys() and img1 in loftr_matches[img2].keys():
            loftr_m = loftr_matches[img2][img1]
        elif super_m is None:
                continue

        loftr_m = {
            "matches0": np.array([]),
            "matching_scores0": np.array([]),
        }

        ensemble_matches[img1][img2]["matches0"] = np.concatenate(
            [
                super_m["matches0"],
                np.array(loftr_m["matches0"]) + ensemble_features[img1]["n_superpoint"],
            ],
            axis=0,
        )

        ensemble_matches[img1][img2]["matching_scores0"] = np.concatenate(
            [
                super_m["matching_scores0"],
                loftr_m["matching_scores0"],
            ],
            axis=0,
        )

In [ ]:
ens_matches_ds = h5.File(os.path.join(matches_dir, "ensemble.h5"), "w")
for img1 in tqdm(ensemble_matches.keys()):
    ens_matches_ds.create_group(img1)
    for img2 in ensemble_matches[img1].keys():
        ens_matches_ds[img1].create_group(img2)
        for k in ensemble_matches[img1][img2].keys():
            ens_matches_ds[img1][img2].create_dataset(
                k, data=ensemble_matches[img1][img2][k]
            )

ens_matches_ds.close()

In [ ]:
from pathlib import Path
from hloc import reconstruction

reconstruction.main(
    sfm_dir=Path(os.path.join(base_dir, scene, "sparse", "ensemble")),
    image_dir=Path(os.path.join(base_dir, scene, "images")),
    pairs=Path(os.path.join(matches_dir, "retrieval", "netvlad-50.txt")),
    features=Path(os.path.join(features_dir, "ensemble.h5")),
    matches=Path(os.path.join(matches_dir, "ensemble.h5")),
    verbose=True,
)